In [1]:
import pandas as pd
import numpy as np
from babel.numbers import format_currency

In [2]:
index_name = "PHGY"
etf_name = "PFF"

# Output File Name
outputFile = 'Output/{} ETF vs ICE.xlsx'.format(etf_name)

# Total market Cap for PFF, needs to be manually updated
# Can be automated later
current_pff_market_cap = 13679614009
current_pff_cash = 90000000

# Reading ICE Data For Projected Universe
projected_universe_df = pd.read_excel('Data/PHGY-Projected.xlsx', skiprows=[0])

# Reading PFF Website Data for Current Shares
pff_current_holdings_df = pd.read_excel('Data/PFF_Website_Data.xlsx')
pff_current_holdings_df = pff_current_holdings_df[['ISIN', 'Shares']]
pff_current_holdings_df.rename(columns = {'ISIN':'ISIN number', 'Shares': 'Current PFF Shares'}, inplace=True)

# DataFrame for our final Output
pff_df = pd.DataFrame()

In [3]:
# When reading data, below strings are read, they must be marked as np.nan
msg1 = 'Any unauthorized use or disclosure is prohibited. Nothing herein should in any way be deemed to alter the legal rights and obligations contained in agreements between any ICE Data Services entity ("ICE") and their clients relating to any of the Indices or products or services described herein. The information provided by ICE and contained herein is subject to change without notice and does not constitute any form of representation, or undertaking.  ICE and its affiliates make no warranties whatsoever, either express or implied, as to merchantability, fitness for a particular purpose, or any other matter in connection with the information provided. Without limiting the foregoing, ICE and its affiliates makes no representation or warranty that any information provided hereunder are complete or free from errors, omissions, or defects. All information provided by ICE is owned by or licensed to ICE. ICE retains exclusive ownership of the ICE Indices, including the ICE BofAML Indexes, and the analytics used to create this analysis ICE may in its absolute discretion and without prior notice revise or terminate the ICE information and analytics at any time. The information in this analysis is for internal use only and redistribution of this information to third parties is expressly prohibited.'
msg2 = 'Neither the analysis nor the information contained therein constitutes investment advice or an offer  or an invitation to make an offer  to buy or sell any securities or any options  futures or other derivatives related to such securities. The information and calculations contained in this analysis have been obtained from a variety of sources  including those other than ICE and ICE does not guarantee their accuracy.  Prior to relying on any ICE information and/or the execution of a security trade based upon such ICE information, you are advised to consult with your broker or other financial representative to verify pricing information. There is no assurance that hypothetical results will be equal to actual performance under any market conditions. THE ICE INFORMATION IS PROVIDED TO THE USERS "AS IS." NEITHER ICE, NOR ITS AFFILIATES, NOR ANY THIRD PARTY DATA PROVIDER WILL BE LIABLE TO ANY USER OR ANYONE ELSE FOR ANY INTERRUPTION, INACCURACY, ERROR OR OMISSION, REGARDLESS OF CAUSE, IN THE ICE INFORMATION OR FOR ANY DAMAGES RESULTING THEREFROM. In no event shall ICE or any of its affiliates, employees  officers  directors or agents of any such persons have any liability to any person or entity relating to or arising out of this information, analysis  or the indices  contained herein.'

In [4]:
# Projected Universe Data
projected_universe_df = projected_universe_df.replace('NaN', np.nan)
projected_universe_df = projected_universe_df.replace(msg1, np.nan)
projected_universe_df= projected_universe_df.replace(msg2, np.nan)

#drop row if ISIN number is Nan
projected_universe_df.dropna(subset=['ISIN number'], inplace=True)

In [5]:
# Storing the unique ISIN ID's from both the current and projected universe data
unique_ISIN = pd.concat([pff_current_holdings_df['ISIN number'], projected_universe_df['ISIN number']]).drop_duplicates().reset_index(drop=True)
pff_df['ISIN number'] = unique_ISIN

In [6]:
isin_to_ticker_df = pd.read_excel('Static Data/ISINtoTicker.xlsx')
pff_df = pff_df.merge(isin_to_ticker_df, left_on='ISIN number', right_on='ISIN', how='left').drop(['ISIN'], axis=1)

In [7]:
def getProjectedMarketCap(x):
    isid = x['ISIN number']
    ticker = projected_universe_df[projected_universe_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['% Mkt Value']
    return 0

pff_df['Projected % Mkt Cap'] = pff_df.apply(getProjectedMarketCap, axis=1)

In [8]:
def getCurrentPFFShares(x):
    isid = x['ISIN number']
    ticker = pff_current_holdings_df[pff_current_holdings_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['Current PFF Shares']
    return 0

pff_df['Current PFF Shares'] = pff_df.apply(getCurrentPFFShares, axis=1)

In [9]:
def getProjectedPFFShares(x):
    if(not isinstance(x['Last Price'], str)):
        return np.rint(((current_pff_market_cap-current_pff_cash)*x['Projected % Mkt Cap'])/(x['Last Price']*100))
    return np.nan

pff_df['Projected PFF Shares'] = pff_df.apply(getProjectedPFFShares, axis=1)
pff_df['Difference'] = pff_df.apply(lambda x: x['Projected PFF Shares']-x['Current PFF Shares'], axis=1)

In [10]:
pff_df.drop(['Activ Ticker', 'Bloomberg File Ticker', 'Activ Last Price Formula'], axis=1, inplace=True)

In [11]:
def getDifference(x):
    if((not isinstance(x['Projected PFF Shares'], str)) and (not isinstance(x['Current PFF Shares'], str))):
        return x['Projected PFF Shares']-x['Current PFF Shares']
    return np.nan

def getDollarDifference(x):
    if((not isinstance(x['Last Price'], str)) and (not isinstance(x['Difference'], str))):
        return x['Last Price']*x['Difference']
    return np.nan

pff_df['Difference'] = pff_df.apply(getDifference, axis=1)
pff_df['Difference $'] = pff_df.apply(getDollarDifference, axis=1)

In [12]:
pff_df.sort_values('Difference')
pff_df.sort_values(by='Difference $', key=abs, ascending=False, inplace=True)
pff_df['Difference $'] = pff_df['Difference $'].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

In [13]:
# Number of Shares
total_buys_ice = pff_df['Difference'].where(pff_df['Difference']>0).sum()
total_sells_ice = pff_df['Difference'].where(pff_df['Difference']<0).sum()

# Dollar value

total_dollar_buying = pff_df[pff_df['Difference']>0].apply(lambda x: x['Last Price']*x['Difference'], axis=1).sum()
total_dollar_selling = pff_df[pff_df['Difference']<0].apply(lambda x: x['Last Price']*x['Difference'], axis=1).sum()

total_pff_transactions_df = pd.DataFrame({" ":[total_buys_ice, 
                                               total_sells_ice,
                                               total_dollar_buying,
                                               total_dollar_selling,
                                               current_pff_market_cap,
                                               current_pff_cash]}, 
                                         
                                         index=['Total PFF Buys (Number of Shares)', 
                                                'Total PFF Sells (Number of Shares)',
                                                'Total PFF Buy (in $)',
                                                'Total PFF Sell (in $)',
                                                "PFF Market Cap",
                                                "PFF Cash"])

In [14]:
with pd.ExcelWriter(outputFile, mode="w") as writer:
    pff_df.to_excel(writer, sheet_name='PFF')
    total_pff_transactions_df.to_excel(writer, sheet_name='Total Transactions - PFF')
    
    workbook = writer.book
    worksheet = writer.sheets['PFF']

    cellFormat = workbook.add_format({'num_format': '#,###'})
    worksheet.set_column('F:H', 10, cellFormat)

#     writer.save()

In [15]:
pff_df.head()